# MTG Format Price Pioneer

In [1]:
import pandas as pd
import requests
import json

Get and clean all the pioneer legal cards from scryfall

In [2]:
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
oracle_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "oracle_cards"').index[0]]

response = requests.get(oracle_uri)
raw_oracle_cards = pd.json_normalize(response.json())

oracle_cards = raw_oracle_cards[[#'object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id',
       #'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 
       'name', 
       #'lang',
       #'released_at', 'uri', 'scryfall_uri', 
       'layout', 
       #'highres_image',
       #'image_status', 'mana_cost', 'cmc', 'type_line', 'oracle_text',
       #'colors', 'color_identity', 'keywords', 'games', 'reserved',
       #'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint',
       #'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri',
       #'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       #'prints_search_uri', 'collector_number', 'digital', 
       #'rarity',
       #'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       #'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       #'booster', 'story_spotlight', 'edhrec_rank', 'image_uris.small',
       #'image_uris.normal', 'image_uris.large', 'image_uris.png',
       #'image_uris.art_crop', 'image_uris.border_crop',
       #'legalities.standard',
       #'legalities.future', 'legalities.historic',
       #'legalities.gladiator', 
       'legalities.pioneer',
       #'legalities.explorer', 'legalities.modern', 'legalities.legacy',
       #'legalities.pauper', 'legalities.vintage', 'legalities.penny',
       #'legalities.commander', 'legalities.brawl',
       #'legalities.historicbrawl', 'legalities.alchemy',
       #'legalities.paupercommander', 'legalities.duel',
       #'legalities.oldschool', 'legalities.premodern', 
       'prices.usd',
       #'prices.usd_foil', 'prices.usd_etched', 'prices.eur',
       #'prices.eur_foil', 
       'prices.tix', 
       #'related_uris.gatherer',
       #'related_uris.tcgplayer_infinite_articles',
       #'related_uris.tcgplayer_infinite_decks', 'related_uris.edhrec',
       #'security_stamp', 'preview.source', 'preview.source_uri',
       #'preview.previewed_at', 'power', 'toughness', 'penny_rank',
       #'arena_id', 'watermark', 'produced_mana', 'all_parts',
       'card_faces', 
       #'frame_effects', 'tcgplayer_etched_id',
       #'promo_types', 'loyalty', 'life_modifier', 'hand_modifier',
       #'attraction_lights', 'color_indicator', 'content_warning'
       ]]
oracle_cards = oracle_cards.rename(columns = {
    'name':'Name',
    'rarity':'Rarity',
    'legalities.pioneer':'Pioneer_Legal',
    'prices.usd':'Price_USD',
    'prices.tix':'Price_Tix'
})

pioneer_raw = oracle_cards.query('Pioneer_Legal == "legal"').sort_values(by =['Name']).reset_index(drop = True)
pioneer_raw.head()

,Name,layout,Pioneer_Legal,Price_USD,Price_Tix,card_faces
0,+2 Mace,normal,legal,0.02,0.02,NaN
1,A Little Chat,normal,legal,0.04,0.01,NaN
2,Abandon Reason,normal,legal,0.12,0.03,NaN
3,Abandon the Post,normal,legal,0.01,0.03,NaN
4,Abandoned Sarcophagus,normal,legal,0.10,None,NaN


In [3]:
#there are split cards included in thi format, we need to fix those in addition to mdfcs, and they work differently because of mtggoldfish's formatting
card_faces = pd.json_normalize(pioneer_raw['card_faces'].loc[~pioneer_raw['card_faces'].isna()])
front_face = pd.json_normalize(card_faces[0])
back_face = pd.json_normalize(card_faces[1])


faces_index = 0 # we lose the relationship between the indices when we normalize the faces data, I'm not sure how to fix this with pandas but just keeping track of it like this works fine
for card_index in pioneer_raw.loc[~pioneer_raw['card_faces'].isna()].index:
    if pioneer_raw['layout'].loc[card_index] == 'split':
        pioneer_raw['Name'].loc[card_index] = front_face['name'].loc[faces_index]+'/'+back_face['name'].loc[faces_index]
    else:
        pioneer_raw["Name"].loc[card_index] = front_face['name'].loc[faces_index]
    faces_index += 1
pioneer = pioneer_raw.drop(['card_faces','layout'],axis=1)
pioneer.head()

,Name,Pioneer_Legal,Price_USD,Price_Tix
0,+2 Mace,legal,0.02,0.02
1,A Little Chat,legal,0.04,0.01
2,Abandon Reason,legal,0.12,0.03
3,Abandon the Post,legal,0.01,0.03
4,Abandoned Sarcophagus,legal,0.10,None


Set up and fill dataframe of decks and decklists

In [4]:
#make df and add names
decks = pd.DataFrame(columns=['Deck_Name','Deck_List','Paper_Price','MTGO_Price_Tix'])
pioneer_deck_names = ['Rakdos Midrange','Abzan Greasefang','Nykthos Ramp','Azorius Control','Temur Indomitable Creativity',
         'Lotus Field Combo','Gruul Midrange','Izzet Phoenix','5 Color Midrange','Selesnya Angels',
         'Mono-White Humans','Dimir Control','Azorius Spirits','5 Color Fires of Invention','Mono-Red Goblins',
         'Mono-Blue Spirits','Rakdos Sacrifice','4 Color Auras','Mono-Black Midrange','Bant Spirits']
decks['Deck_Name'] = pioneer_deck_names

#read in the decklists
pioneer_deck_decklists = []
for deck in pioneer_deck_names:
    pioneer_deck_decklists.append(pd.read_csv('Deck Lists/Pioneer Metagame Decks/'+deck+'.csv'))

decks['Deck_List'] = pioneer_deck_decklists

decks[['Deck_Name','Deck_List']].head()

,Deck_Name,Deck_List
0,Rakdos Midrange,Quantity Card_Name 0 ...
1,Abzan Greasefang,Quantity Card_Name 0 ...
2,Nykthos Ramp,Quantity Card_Name 0 ...
3,Azorius Control,Quantity Card_Name 0 ...
4,Temur Indomitable Creativity,Quantity Card_Name 0...


Define, call, and assign outputs for price calculating functions

In [5]:
def paper_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in range(len(decklist.index)): #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = pioneer.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if pioneer['Price_USD'].loc[price_index]:
            card_price = float(pioneer['Price_USD'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

In [6]:
def mtgo_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in range(len(decklist.index)): #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = pioneer.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if pioneer['Price_Tix'].loc[price_index]:
            card_price = float(pioneer['Price_Tix'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

In [7]:
for i in decks.index:
    decks['Paper_Price'].loc[i] = paper_price(decks['Deck_List'].loc[i])
    decks['MTGO_Price_Tix'].loc[i] = mtgo_price(decks['Deck_List'].loc[i])

decks.head(len(decks))

,Deck_Name,Deck_List,Paper_Price,MTGO_Price_Tix
0,Rakdos Midrange,Quantity Card_Name 0 ...,696.06,529.14
1,Abzan Greasefang,Quantity Card_Name 0 ...,509.28,325.15
2,Nykthos Ramp,Quantity Card_Name 0 ...,492.94,249.58
3,Azorius Control,Quantity Card_Name 0 ...,339.14,174.49
4,Temur Indomitable Creativity,Quantity Card_Name 0...,594.83,387.53
5,Lotus Field Combo,Quantity Card_Name 0 ...,348.27,235.93
6,Gruul Midrange,Quantity Card_Name 0...,317.53,204.69
7,Izzet Phoenix,Quantity Card_Name 0 ...,386.96,201.65
8,5 Color Midrange,Quantity Card_Name 0...,539.66,467.67
9,Selesnya Angels,Quantity Card_Name 0 ...,554.56,225.49


Output

In [9]:
decks[['Deck_Name','Paper_Price','MTGO_Price_Tix']].to_csv('Format Price Data/Pioneer_Metagame_Deck_Prices.csv',index=False)